In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import imutils
from imutils import paths
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import cv2

In [7]:
import os

root = "C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects"
tree = list(os.walk(root))
folders = tree[1:]
imgPaths = []
for sub in folders:
    root = sub[0].replace("\\", "/")
    for img in sub[2]:
        imgPaths.append(root + '/' + img)

print(len(imgPaths))
for path in imgPaths[:60]:
    print(path)


9146
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0001.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0002.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0003.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0004.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0005.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0006.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0007.jpg
C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB I

In [8]:
import pickle as pk
pk.dump(imgPaths, open(outputDestination + "/imgPaths.pkl","wb"))

In [24]:
fe = FeatureExtractor()

In [25]:
import csv
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [3]:
outputDestination = "C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/img_engine"

In [27]:
# IMG_PER_FOLDER = 10000
DATASET_SIZE = 9146
done = 0


with open(outputDestination + '/vggOutputs.csv', mode='w', newline='') as vggOutputs:
    outputWriter = csv.writer(vggOutputs, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # iterate over each image within each folder and generate its feature vector using vgg16 algorithm 
    for path in imgPaths:
        img = Image.open(path)
        with suppress_stdout():
            imgFeatures = fe.extract(img)
        if len(imgFeatures) != 4096:
            print("{} have size {} =/= 4096".format(path))
            assert False
        outputWriter.writerow(imgFeatures)
        done += 1
        print("[{}/{}] => {:.2f}% Done!             ".format(done, DATASET_SIZE, done/DATASET_SIZE * 100), end = '\r') # dul chun cinn All Rights Reserved.


In [28]:
import pandas as pd

with open(outputDestination + '/vggOutputs.csv') as input:
    vggFeatures = list(csv.reader(input, delimiter=','))

vggFeatures = np.array(vggFeatures)


In [29]:
vggFeatures.shape

(9146, 4096)

In [30]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
dtReducedFeatures = pca.fit_transform(vggFeatures)
pca.tra

In [39]:
done = 0

with open(outputDestination + '/reducedVggOutputs.csv', mode='w', newline='') as reducedVggOutputs:
    outputWriter = csv.writer(reducedVggOutputs, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # iterate over each image within each folder and generate its feature vector using vgg16 algorithm 
    for featureVect in dtReducedFeatures:
        outputWriter.writerow(featureVect)
        done += 1
        print("[{}/{}] => {:.2f}% Done!             ".format(done, DATASET_SIZE, done/DATASET_SIZE * 100), end = '\r') # dul chun cinn All Rights Reserved.


In [40]:
import pickle as pk
pk.dump(pca, open(outputDestination + "/pca.pkl","wb"))

In [ ]:
pca_reload = pk.load(open("pca.pkl",'rb'))

In [5]:
import pandas as pd
import csv
with open(outputDestination + '/reducedVggOutputs.csv') as input:
    vggFeatures = list(csv.reader(input, delimiter=','))

vggFeatures = np.array(vggFeatures)

In [6]:
vggFeatures.shape

(9146, 2209)

In [90]:

# Test 
url = paths[0] + "/im1.jpg"
img = Image.open(url)
imgFeatures = fe.extract(img)
reducedFeatures = pca.transform(np.array([list(imgFeatures)]))
hit = min([(i,dtSample) for i, dtSample in enumerate(dtReducedFeatures)], key = lambda element: np.linalg.norm(element[1]-reducedFeatures, axis=None))
print("difference = ", np.linalg.norm(dtReducedFeatures[0]-reducedFeatures, axis=None))
print("(position, found hit) = ", hit)

1/1 [==============================] - 0s 468ms/step
difference =  1.6029194721527013e-08
(position, found hit) =  (0, array([-0.10915537, -0.11047512, -0.14579126, ..., -0.00195563,
       -0.00237214,  0.0026641 ]))


In [78]:
vggFeatures.iloc[0]

(4096,)